In [1]:
import pandas as pd
import numpy as np
import glob
import datetime

In [3]:
mex_violence = pd.read_csv('../data/icews_mexico_violence.csv')

In [4]:
mex_violence.head(10)

,Unnamed: 0,event_id,event_date,source_name,source_sectors,source_country,event_text,cameo_code,intensity,target_name,...,story_id,sentence_number,publisher,city,district,provice,country,lat,lon,event_year
0,325,927232,1995-01-06,Citizen (Mexico),"Social,General Population / Civilian / Social",Mexico,Occupy territory,192,-9.5,Mexico,...,28238835,1,The Associated Press Political Service,Amatenango del Valle,Amatenango del Valle,Estado de Chiapas,Mexico,16.5167,-92.4500,1995
1,326,927233,1995-01-06,Citizen (Mexico),"Social,General Population / Civilian / Social",Mexico,Occupy territory,192,-9.5,Government (Mexico),...,28238835,4,The Associated Press Political Service,Amatenango del Valle,Amatenango del Valle,Estado de Chiapas,Mexico,16.5167,-92.4500,1995
2,336,927250,1995-01-06,Armed Band (Brazil),Unidentified Forces,Brazil,fight with small arms and light weapons,193,-10.0,Criminal (Brazil),...,28241498,5,Reuters News,El Rio,NaN,Estado de San Luis Potosi,Mexico,22.0020,-100.5150,1995
3,1351,928757,1995-01-19,Party Member (Mexico),Parties,Mexico,Use conventional military force,190,-10.0,Citizen (Mexico),...,28240215,1,The Associated Press Political Service,Villahermosa,NaN,Distrito Federal,Mexico,19.4833,-99.1000,1995
4,1436,928894,1995-01-20,Institutional Revolutionary Party,"Parties,(National) Major Party,Ideological,Cen...",Mexico,Use conventional military force,190,-10.0,Opposition Activist (Mexico),...,28240736,2,The Associated Press Political Service,Villahermosa,NaN,Distrito Federal,Mexico,19.4833,-99.1000,1995
5,2941,931052,1995-02-03,Attacker (Mexico),"Dissident,Criminals / Gangs",Mexico,fight with small arms and light weapons,193,-10.0,Senate (Mexico),...,28244924,1,The Associated Press Political Service,Mexico City,NaN,Distrito Federal,Mexico,19.4285,-99.1277,1995
6,9984,940649,1995-04-08,Criminal (Mexico),"Criminals / Gangs,Dissident",Mexico,Use unconventional violence,180,-9.0,Citizen (Mexico),...,28216851,1,The Washington Post,Tijuana,NaN,Estado de Baja California,Mexico,32.5027,-117.0040,1995
7,11321,942598,1995-04-25,Democratic Revolutionary Party,"(National) Major Party,Parties,Center Left,Ide...",Panama,Use conventional military force,190,-10.0,Citizen (Mexico),...,28214321,2,The Associated Press Political Service,Morelia,NaN,Estado de Chiapas,Mexico,16.5828,-93.7818,1995
8,12660,944443,1995-05-05,Police (Mexico),"Government,Police",Mexico,Use conventional military force,190,-10.0,Criminal (Mexico),...,28920722,1,Reuters News,Mexico City,NaN,Distrito Federal,Mexico,19.4285,-99.1277,1995
9,12661,944444,1995-05-05,Criminal (Mexico),"Dissident,Criminals / Gangs",Mexico,Use unconventional violence,180,-9.0,Police (Mexico),...,28920722,1,Reuters News,Mexico City,NaN,Distrito Federal,Mexico,19.4285,-99.1277,1995


In [5]:
mex_violence["event_text"].value_counts()

Use unconventional violence                            7545
Use conventional military force                        4136
fight with small arms and light weapons                3036
Abduct, hijack, or take hostage                        1353
Physically assault                                      450
Occupy territory                                        384
Assassinate                                             207
Kill by physical assault                                149
Torture                                                 148
Conduct suicide, car, or other non-military bombing     142
Impose blockade, restrict movement                      136
Defy norms, law                                         133
Engage in mass killings                                  49
Employ aerial weapons                                    19
fight with artillery and tanks                            4
Use as human shield                                       3
Attempt to assassinate                  

In [ ]:
#standardize cartel events
cartels = {
    "Armed Gang (Sinaloa Cartel)": 'Sinaloa Cartel',
    "Armed Gang (Juárez Cartel)":'Juárez Cartel',
    "Organized Crime (Gulf Cartel)":'Gulf Cartel',
    "Armed Gang (Gulf Cartel)":'Gulf Cartel',
    "Drug Gang (Los Zetas Cartel)":'Los Zetas Cartel',
    "Hitman (Gulf Cartel)":'Gulf Cartel',
    "Hitman (Los Zetas Cartel)":'Los Zetas Cartel',
    "Criminal (Los Zetas Cartel)":'Los Zetas Cartel',
    "Drug Gang (Gulf Cartel)":'Gulf Cartel',
    "Armed Services Deserter (Los Zetas Cartel)":'Los Zetas Cartel',
    "Armed Gang (Knights Templar Cartel)":'Knights Templar Cartel'
}
mex_violence_v1=mex_violence.replace(cartels)